# Data analysis

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager
from urllib.parse import urlparse
import urllib
import json
import pandas as pd
import os
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 96.0.4664
[WDM] - Get LATEST driver version for 96.0.4664
[WDM] - Driver [/home/marco/.wdm/drivers/chromedriver/linux64/96.0.4664.45/chromedriver] found in cache
/tmp/ipykernel_3948468/3270087180.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [2]:
driver.get("https://sarc.asystom.com/monitor/login")
# import time
# time.sleep(1)
elem = driver.find_element_by_name("user")
elem.clear()
elem.send_keys(os.environ["SITE_USER"])

elem = driver.find_element_by_name("password")
elem.clear()
elem.send_keys(os.environ["SITE_PASSWORD"])

btn = driver.find_element_by_class_name("css-mij01a-button")
btn.click()

/tmp/ipykernel_3948468/3561746922.py:4: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  elem = driver.find_element_by_name("user")
/tmp/ipykernel_3948468/3561746922.py:8: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  elem = driver.find_element_by_name("password")
/tmp/ipykernel_3948468/3561746922.py:12: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  btn = driver.find_element_by_class_name("css-mij01a-button")


In [3]:
def get_dfs(parsed_json):
    dfs = []
    # parsed_json["results"][0]["series"][0]["columns"]
    # parsed_json["results"][0]["series"][0]["values"]

    for result in parsed_json["results"]:
        print("Result", result["statement_id"])
        for serie in result["series"]:
            df = pd.DataFrame(serie["values"], columns=serie["columns"])
            df.name = serie["name"]
            dfs.append(df)
    return dfs

In [4]:
from urllib import parse
q="SELECT%20median(%22drift%22)%20FROM%20(SELECT%2020*log10(%22vib_x_vel%22*%22vib_y_vel%22*%22vib_z_vel%22)%20AS%20%22drift%22%20FROM%20%22Signature_Drift%22%20WHERE%20%22device%22%3D%2700-04-a3-0b-00-e9-b5-05%27%20AND%20time%20%3E%3D%20now()%20-%2030d%20AND%20%22mileage%22%252%3D1)%20GROUP%20BY%20time(2h)%20fill(none)%3BSELECT%20moving_average(%22decimated%22%2C%2050)%20FROM%20(SELECT%20median(%22drift%22)%20AS%20%22decimated%22%20FROM%20(SELECT%2020*log10(%22vib_x_vel%22*%22vib_y_vel%22*%22vib_z_vel%22)%20AS%20%22drift%22%20FROM%20%22Signature_Drift%22%20WHERE%20%22device%22%3D%2700-04-a3-0b-00-e9-b5-05%27%20AND%20time%20%3E%3D%20now()%20-%2030d%20AND%20%22mileage%22%252%3D1)%20GROUP%20BY%20time(1m)%20fill(none))"

q="https://sarc.asystom.com/monitor/api/datasources/proxy/8/query?db=asystom_db&q=SELECT%20last(%22drift%22)%20FROM%20(SELECT%2020*log10(%22vib_x_vel%22*%22vib_y_vel%22*%22vib_z_vel%22)%20AS%20%22drift%22%20FROM%20%22Signature_Drift%22%20WHERE%20%22device%22%3D%2700-04-a3-0b-00-e9-b5-05%27%20AND%20time%20%3E%3D%20now()%20-%201d%20AND%20%22mileage%22%252%3D1)%0A&epoch=ms"
q_clean = parse.unquote(q)
q_clean

'https://sarc.asystom.com/monitor/api/datasources/proxy/8/query?db=asystom_db&q=SELECT last("drift") FROM (SELECT 20*log10("vib_x_vel"*"vib_y_vel"*"vib_z_vel") AS "drift" FROM "Signature_Drift" WHERE "device"=\'00-04-a3-0b-00-e9-b5-05\' AND time >= now() - 1d AND "mileage"%2=1)\n&epoch=ms'

In [6]:
# SELECT vib_x_vel, vib_y_vel, vib_z_vel

delay = "200d"

q_clean = f"""
SELECT *
FROM "Signature_Drift"
WHERE "device"=\'00-04-a3-0b-00-e9-b5-05\' AND time >= now() - {delay};

SELECT *
FROM "Temperature"
WHERE "device"=\'00-04-a3-0b-00-e9-b5-05\' AND time >= now() - {delay};

SELECT *
FROM "Current_Loop"
WHERE "device"=\'00-04-a3-0b-00-e9-b5-05\' AND time >= now() - {delay};
"""

driver.set_page_load_timeout(600) # 10 minutes

q_econded = parse.quote(q_clean)
url = f'https://sarc.asystom.com/monitor/api/datasources/proxy/8/query?db=asystom_db&q={q_econded}&epoch=ms'
print("Opening:", url)
driver.get(url)
driver.page_source
content = driver.find_element_by_tag_name('pre').text
parsed_json = json.loads(content)
parsed_json
dfs = get_dfs(parsed_json)

Opening: https://sarc.asystom.com/monitor/api/datasources/proxy/8/query?db=asystom_db&q=%0ASELECT%20%2A%0AFROM%20%22Signature_Drift%22%0AWHERE%20%22device%22%3D%2700-04-a3-0b-00-e9-b5-05%27%20AND%20time%20%3E%3D%20now%28%29%20-%20200d%3B%0A%0ASELECT%20%2A%0AFROM%20%22Temperature%22%0AWHERE%20%22device%22%3D%2700-04-a3-0b-00-e9-b5-05%27%20AND%20time%20%3E%3D%20now%28%29%20-%20200d%3B%0A%0A%0ASELECT%20%2A%0AFROM%20%22Current_Loop%22%0AWHERE%20%22device%22%3D%2700-04-a3-0b-00-e9-b5-05%27%20AND%20time%20%3E%3D%20now%28%29%20-%20200d%3B%0A&epoch=ms


/tmp/ipykernel_3948468/3711297532.py:27: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  content = driver.find_element_by_tag_name('pre').text


Result 0
Result 1
Result 2


In [7]:
with open("all_data.json", "w") as f:
    json.dump(parsed_json, f)

In [8]:
print(dfs[0].name)
dfs[0]

Signature_Drift


,time,GW,client,device,mileage,s_00,s_01,s_02,s_03,s_04,...,vib_y_vel,vib_z_acc,vib_z_f1,vib_z_f2,vib_z_f3,vib_z_kurt,vib_z_peak,vib_z_root,vib_z_vel,vibra_custom
0,1632822027177,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,258,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,1.0,1.0,1.0,1.000000,1.000000,1.0,1.000000,1
1,1632822331552,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,259,0.603113,0.428016,1.520943,0.162509,0.070954,...,1.057239,1.102804,1.0,1.0,1.0,0.930266,0.992188,1.0,1.152778,1
2,1632822626555,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,259,1.068894,0.929274,1.863890,1.574731,0.604257,...,1.106270,1.089286,1.0,1.0,1.0,1.433232,1.603636,1.0,1.103600,1
3,1632822926402,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,259,0.295262,-0.921836,-0.814832,0.993935,0.504692,...,1.054773,1.013333,1.0,1.0,1.0,1.080572,1.076412,1.0,1.035784,1
4,1632823226427,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,259,0.441291,-0.576333,1.248798,2.314946,0.782788,...,1.028589,1.066434,1.0,1.0,1.0,1.477110,1.229960,1.0,1.038660,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31732,1638464014032,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,512,-13.453398,-6.449507,-1.423546,-0.086615,-0.523545,...,0.290588,0.942460,1.0,1.0,1.0,0.981932,0.907035,1.0,0.763052,1
31733,1638464134034,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,512,-13.055137,-5.975714,-0.611003,-0.738938,0.609437,...,0.281647,0.942460,1.0,1.0,1.0,0.964262,0.897487,1.0,0.839357,1
31734,1638464254034,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,512,-13.416776,-5.982581,-1.462457,-0.299478,-0.399947,...,0.290588,0.942460,1.0,1.0,1.0,0.990009,0.907035,1.0,0.671486,1
31735,1638464374044,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,513,8.535616,-5.991736,-1.373192,0.121671,-0.232861,...,2.208471,3.279762,1.0,1.0,1.0,0.987990,2.625628,1.0,11.888353,1


In [9]:
print(dfs[1].name)
dfs[1]

Temperature


,time,GW,client,device,value,weekday
0,1630308787973,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,22.305737,Monday
1,1630308847953,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,22.305737,Monday
2,1630308868190,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,22.340423,Monday
3,1630308927926,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,22.340423,Monday
4,1630308987909,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,22.340423,Monday
...,...,...,...,...,...,...
53547,1638464193881,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,14.154516,Thursday
53548,1638464254034,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,14.119830,Thursday
53549,1638464313887,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,14.119830,Thursday
53550,1638464374044,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,14.085144,Thursday


In [10]:
print(dfs[2].name)
dfs[2]

Current_Loop


,time,GW,client,device,value,weekday
0,1630308787973,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,0.000458,Monday
1,1630308847953,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,0.000458,Monday
2,1630308868190,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,0.000916,Monday
3,1630308927926,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,0.000916,Monday
4,1630308987909,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,0.000458,Monday
...,...,...,...,...,...,...
53547,1638464193881,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,0.001373,Thursday
53548,1638464254034,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,0.000916,Thursday
53549,1638464313887,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,0.001373,Thursday
53550,1638464374044,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,0.000916,Thursday
